In [1]:
#import data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

books = pd.read_csv("/opt/ml/input/code/data/books.csv")
users = pd.read_csv("/opt/ml/input/code/data/users.csv") # books.drop(['img_url', 'img_path'], axis = 1, inplace = True)
isbns = pd.read_csv("/opt/ml/input/code/data/isbn_country.csv")

## Users 결측치

### Users location 세분화

In [2]:
# users 결측치
### location 세분화 -> city, state, county 로
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.
###

/tmp/ipykernel_51356/2406416191.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


### Users location city만 있는 경우 처리

In [3]:
# users 결측치
### city값이 존재하는데, country 정보가 없는 경우 처리
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
    
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]

In [4]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       1132
location_country      271
dtype: int64

### User age 처리

In [5]:
# user의 age를 평균으로 채움
users.age = users.age.fillna(users.age.mean())

In [6]:
users.isna().sum()

user_id                0
location               0
age                    0
location_city        122
location_state      1132
location_country     271
dtype: int64

In [7]:
# location에서 결측값은 'na' 문자열로 채움
users['location_city'].fillna('na', inplace=True)
users['location_state'].fillna('na', inplace=True)
users['location_country'].fillna('na', inplace=True)

In [10]:
users.isna().sum()

user_id             0
location            0
age                 0
location_city       0
location_state      0
location_country    0
dtype: int64

# Books 결측치

In [11]:
# 잘못된 isbn 확인
books[books['isbn'] != books['img_url'].apply(lambda x: x.split('P/')[1][:10])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


In [12]:
books[books['isbn'] != books['img_path'].apply(lambda x: x.split('.')[0][7:])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


### Category 대분류 생성

In [13]:
# category 대분류

import re
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
books['category'] = books['category'].str.lower()

books['category_low'] = books['category'].copy()

category_high_df = pd.DataFrame(books['category'].value_counts()).reset_index()
category_high_df.columns = ['category', 'count']

other_list = category_high_df[category_high_df['count'] < 5]['category']
books.loc[books[books['category'].isin(other_list)].index, 'category'] = 'others'

books.loc[books[books['category']=='biography'].index, 'category'] = 'biography autobiography'
books.loc[books[books['category']=='autobiography'].index,'category'] = 'biography autobiography'

categories = ['history', 'fiction', 'garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category'] = category

### 데이터 중 특수문자(\&#39;, 문자('\n')을 공백(' ') 전처리)

In [14]:
# summary에 특수문자가 &#39;로 표시된 경우 처리
# 개행 문자('\n')을 공백(' ')으로 변경
summary_ = pd.DataFrame(books[['book_title','summary']])

dic = {
    "&quot;" : '"',
    "&amp;" : "&",
    "&lt;" : "<",
    "&gt;" : '>',
    "&#34;" : '"',
    '&#39;' : "'"
}

for k, v in dic.items():
    summary_['summary'] = summary_['summary'].str.replace(k,v, regex=True)
summary_['summary'] = summary_['summary'].str.replace('\n', ' ')
books['summary'] = summary_['summary']

### book_author 공백 없이 보게 전처리

In [15]:
# book_author 모두 소문자 -> 공백 제거 후 모두 이어붙임
book_author = books['book_author'].str.lower()
books['book_author'] = book_author.map(lambda x: "".join(x.split()))
# books['book_author'].value_counts()

### publisher isbn 기준으로 중복처리

In [16]:
# 같은 출판사인데 이름이 다른 경우 처리 (예. a_book, a_audio)
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
books['publisher'].nunique() # 수정전 항목 수를 확인
books[books['isbn'].apply(lambda x: x[:4])=='0140']['publisher'].unique()
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values
books['publisher'].nunique() #수정 후 출판사 개수

11571

### 작가가 썼던 책을 기준으로 language 채우기

In [17]:
# language 결측치 채우기
# 작가가 쓴 책을 찾아서 language 채우기 (한 작가는 같은 언어로 책을 쓴다고 가정)
look_up_df = books[books.language.notnull()].drop_duplicates(['book_author'])
look_up_df = look_up_df.set_index('book_author')['language']
books['language'] = books['language'].fillna(books['book_author'].map(look_up_df))
books=books.reset_index(drop=True)
books.isna().sum()


isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               25349
category               68851
summary                67227
img_path                   0
category_low           68851
dtype: int64


#### 책제목과 작가가 같으면 같은 책이므로 language가 있는 다른 index에서 language를 가져옴.

In [20]:
# category&summary 결측치 채우기
# 책제목과 작가가 같으면 같은 책이므로 language가 있는 다른 index에서 language를 가져옴.

def new_func(idx_n):
    return books.book_title.isin([books.iloc[idx_n].book_title]) & books.book_author.isin([books.iloc[idx_n].book_author])

for idx_n in books[books.category.isna()].index:
    # print(idx_n)
    books[new_func(idx_n)].ffill().bfill()
    books[new_func(idx_n)] = books[new_func(idx_n)].ffill().bfill()

In [23]:
# 확인
books[books.book_title.isin([books.iloc[25].book_title]) & books.book_author.isin([books.iloc[25].book_author])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_low
25,0312978383,Winter Solstice,rosamundepilcher,2001.0,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312978383.0...,en,NaN,NaN,images/0312978383.01.THUMBZZZ.jpg,NaN
7264,0312244266,Winter Solstice,rosamundepilcher,2000.0,Thomas Dunne Books,http://images.amazon.com/images/P/0312244266.0...,en,fiction,Five people buffeted by life's difficulties co...,images/0312244266.01.THUMBZZZ.jpg,fiction
79374,B00009NDAN,Winter Solstice,rosamundepilcher,2000.0,St. Martin's Press,http://images.amazon.com/images/P/B00009NDAN.0...,en,NaN,NaN,images/B00009NDAN.01.THUMBZZZ.jpg,NaN


### Category 결측값 fiction으로 채우기

In [24]:
# books['category'] 결측치
## fiction이 카테고리 50프로 이상이므로 결측치를 fiction으로 채움
books['category'] = books['category'].fillna('fiction')

In [25]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               25349
category                   0
summary                67227
img_path                   0
category_low           68851
dtype: int64

### isbn 이용해 country 채우기

In [26]:
for i in range(len(isbns)):
    books.loc[books["isbn"].str.startswith(str(isbns.iloc[i][0])), "country"] = isbns.iloc[i][1]

In [27]:
# isbn으로 나라 못찾는 경우는 제거
books = books[~(books.country.isna())]
books.country.dropna(inplace=True)

### language 채우기

In [29]:
books['language'].value_counts() * 100 / len(books[~books['language'].isna()])
# ca: 캐나다식 영어

en       95.156952
de        1.741049
es        1.454364
fr        1.178147
it        0.201324
nl        0.095830
da        0.053150
pt        0.042681
ca        0.033017
ms        0.008858
la        0.008053
no        0.004832
gl        0.004026
zh-CN     0.003221
ja        0.003221
ru        0.002416
eo        0.001611
th        0.000805
el        0.000805
ro        0.000805
fa        0.000805
gd        0.000805
ga        0.000805
vi        0.000805
zh-TW     0.000805
ko        0.000805
Name: language, dtype: float64

In [30]:
len(books['language'].value_counts())

26

In [31]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               25343
category                   0
summary                67179
img_path                   0
category_low           68803
country                    0
dtype: int64

In [32]:
pd.DataFrame(books['country'].value_counts()*100/len(books)).head()

,country
English,89.890383
German,4.484989
French,2.277272
Spain,2.273259
Italian,0.322363


In [34]:
# 영어권 국가에서 발행된 책은 language를 1로, 나머지는 0으로 채움
books['language'] = books['country'].apply(lambda count: 1 if count.strip() == 'English' else 0)

In [35]:
books['language'].sum() * 100 / len(books)

89.89038329064145

In [36]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language                   0
category                   0
summary                67179
img_path                   0
category_low           68803
country                    0
dtype: int64

### summary 채우기

In [37]:
# summary는 book_title로 채움
books.loc[books['summary'].isna(), "summary"] = books.book_title

In [39]:
# summary에 특수문자가 &#39;로 표시된 경우 처리
# 개행 문자('\n')을 공백(' ')으로 변경
summary_ = pd.DataFrame(books[['book_title','summary']])

dic = {
    "&quot;" : '"',
    "&amp;" : "&",
    "&lt;" : "<",
    "&gt;" : '>',
    "&#34;" : '"',
    '&#39;' : "'"
}

for k, v in dic.items():
    summary_['summary'] = summary_['summary'].str.replace(k,v, regex=True)
summary_['summary'] = summary_['summary'].str.replace('\n', ' ')
books['summary'] = summary_['summary']

In [42]:
books.drop('category_low', axis=1, inplace=True)
books.drop('country', axis=1, inplace=True)

In [50]:
books.isna().sum()

isbn                   0
book_title             0
book_author            0
year_of_publication    0
publisher              0
img_url                0
language               0
category               0
summary                0
img_path               0
dtype: int64

In [49]:
# book_title에 특수문자가 &#39;로 표시된 경우 처리
for k, v in dic.items():
    summary_['book_title'] = summary_['book_title'].str.replace(k,v, regex=True)
summary_['book_title'] = summary_['book_title'].str.replace('\n', ' ')
books['book_title'] = summary_['book_title'].apply(lambda x: x.lower())

# csv 파일로 저장

In [48]:
users.to_csv('new_users.csv', index=False)
books.to_csv('new_books.csv', index=False)